In [ ]:
!pip install ../input/effnetpytorchmodel/efficientnet_pytorch-0.1.0-py3-none-any.whl

<h1 style='background:#2cab6c; border:0; color:white'><center>Importing Libraries</center></h1>

In [ ]:
import os
import pandas as pd
import cv2 as cv

import torch
import torch.nn as nn
import efficientnet_pytorch

from torch.utils.data import DataLoader, Dataset

from pathlib import Path

from albumentations import (
    Compose, Normalize
)
from albumentations.pytorch import ToTensorV2

In [ ]:
# Set device
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
# Set configuration
class Config:
    cfg = {
        'batch_size': 32,
        'num_workers': 8,
        'image_size': (512, 512),
        'num_classes': 5,
        'model_path': '../input/effnetb0f3/3_fold_model_effnet_b0_best.torch'
    }

<h1 style='background:#2cab6c; border:0; color:white'><center>Paths, files</center></h1>

In [ ]:
base_dir = Path('/kaggle/input/cassava-leaf-disease-classification')
test_img_dir = f'{base_dir}/test_images'
test_df = pd.read_csv(f'{base_dir}/sample_submission.csv', index_col=0)

In [ ]:
test_df

<h1 style='background:#2cab6c; border:0; color:white'><center>Data Preparation</center></h1>

In [ ]:
class CassavaDataset(Dataset):
    def __init__(self, df, image_size, augments=None):
        self.df = df.index.tolist()
        self.image_size = image_size
        self.augments = augments
        
    def __getitem__(self, idx):
        image = cv.imread(os.path.join(test_img_dir, self.df[idx]))
        image = cv.resize(image, self.image_size)
        image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
        
        if self.augments:
            image = self.augments(image=image)['image']
        
        return {'X': image}
    
    def __len__(self):
        return len(self.df)

In [ ]:
class Augments:
    test_augments = Compose([
        Normalize(mean=[.485, .456, .406],
                  std=[.229, .224, .225],
                  p=1.),
        ToTensorV2(p=1.),
    ],
    p=1.,
    )

In [ ]:
test_dataset = CassavaDataset(df=test_df, image_size=Config.cfg['image_size'], augments=Augments.test_augments)

In [ ]:
test_dataloader = DataLoader(
    test_dataset,
    batch_size=Config.cfg['batch_size'],
    shuffle=False,
    num_workers=Config.cfg['num_workers']
)

<h1 style='background:#2cab6c; border:0; color:white'><center>Model Initialization</center></h1>

In [ ]:
def efficientnet_b0(num_classes):
    model = efficientnet_pytorch.EfficientNet.from_name('efficientnet-b0')
    model._fc = nn.Linear(in_features=1280,
                          out_features=num_classes,
                          bias=True)
    return model

In [ ]:
model = efficientnet_b0(Config.cfg['num_classes']).to(device)

In [ ]:
checkpoint = torch.load(Config.cfg['model_path'], map_location=device)

In [ ]:
model.load_state_dict(checkpoint['model_state_dict'])

best_score = checkpoint['best_valid_score']
epoch_num = checkpoint['num_epoch']

print(f'Best validation score: {round(best_score, 4)} in {epoch_num} epoch.')

<h1 style='background:#2cab6c; border:0; color:white'><center>Evaluation</center></h1>

In [ ]:
model.eval()

In [ ]:
y_prediction = []

for batch in test_dataloader:
    with torch.no_grad():
        X_test = batch['X'].to(device)
        
        y_prediction.extend(model(X_test).argmax(axis=-1).cpu().numpy())
        
print(y_prediction)

<h1 style='background:#2cab6c; border:0; color:white'><center>Submission</center></h1>

In [ ]:
test_df['label'] = y_prediction
test_df.to_csv('submission.csv')